In [36]:
# Librerias, archivos y configuración

import warnings
import os
import polars as pl

warnings.filterwarnings('ignore')

In [37]:
# Primero se unen todos los archivos de los distintos meses en uno solo

# Se hace una lista con los archivos de los distintos meses para poder llamar a la función

archivos = os.listdir('../original_pollution_data')
archivos.sort(reverse=True)

lista_meses = [e for e in archivos]

In [38]:
# Se crea el df

dfs = []

for e in lista_meses:   

        df = pl.read_csv(f'../original_pollution_data/{e}', sep=';')
    
        dfs.append(df)

df = pl.concat(dfs)

In [39]:
df.head()

PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,H02,V02,H03,V03,H04,V04,H05,V05,H06,V06,H07,V07,H08,V08,H09,V09,H10,V10,H11,V11,H12,V12,H13,V13,H14,V14,H15,V15,H16,V16,H17,V17,H18,V18,H19,V19,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
i64,i64,i64,i64,str,i64,i64,i64,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str
28,79,4,1,"""28079004_1_38""",2022,12,1,1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V"""
28,79,4,1,"""28079004_1_38""",2022,12,2,1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",2.0,"""V""",2.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",3.0,"""V""",2.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V"""
28,79,4,1,"""28079004_1_38""",2022,12,3,1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",2.0,"""V""",2.0,"""V""",2.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",2.0,"""V""",1.0,"""V""",1.0,"""V""",2.0,"""V""",2.0,"""V""",2.0,"""V""",1.0,"""V""",2.0,"""V"""
28,79,4,1,"""28079004_1_38""",2022,12,4,2.0,"""V""",2.0,"""V""",2.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V"""
28,79,4,1,"""28079004_1_38""",2022,12,5,1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",2.0,"""V""",3.0,"""V""",2.0,"""V""",1.0,"""V""",1.0,"""V""",2.0,"""V""",2.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V"""


In [40]:
df.shape

(646741, 56)

In [41]:
# Se ponen los nombres de las columnas en minúsculas por comodidad

df.columns = [col.lower() for col in df.columns]

In [42]:
# Es siempre la misma provincia y municipio y el punto de muestre es redundante pues se tienen datos sobre la estacion, por lo que se borran estos datos

df = df.drop(['provincia', 'municipio', 'punto_muestreo'])

In [43]:
''' Todas las columnas del estilo v01 muestran si los datos están validados o no. Se borran puesto que hay muy pocos no 
validados(30 de 3960 para noviembre de 2022) y además se considera que los datos importan aunque no esten validados '''

for e in df.columns:

    if 'v' in e: df = df.drop(e)
    else: pass

In [44]:
# Las columnas de la hora se cambian por la hora como id del 01 al 24

for e in df.columns:
    
    if 'h' in e: df = df.rename({e: e[1:]})
    else: pass

In [45]:
# Se quiere tener una columna con la fecha en formato Year-Month-Day hour:minute:second

# Primero se crea una columna juntando año, mes y día

fechas = []

for i in range(len(df)):

    fechas.append(f"{df.get_column('ano')[i]}-{df.get_column('mes')[i]}-{df.get_column('dia')[i]}")


df = df.with_columns(pl.Series(fechas).alias('fecha'))

df = df.drop(['ano', 'mes', 'dia'])

In [46]:
# Ahora se junta la hora con la fecha

# Se repite cada fila 24 veces

df = pl.concat([df.clone() for _ in range(24)])

In [47]:
# Se quitan los 0 de los nombres de las columnas

df.columns = [e[1:] if e[0] == '0' else e for e in df.columns]

# Se cambia el nombre de la columna 24 por 00

df = df.rename({'24': '0'})


In [48]:
# Se borran las horas que no se quieren en cada fila

horas = []

for i in range(0, len(df), 24):

    for e in range(24):

        horas.append(i+e)

In [49]:
df = df.with_columns(pl.Series(horas).alias('horas'))

In [51]:
valores = []

for i in range(len(df)):

    valores.append(df.with_column(f"{df.with_column('horas')[i]}")[i])

ColumnNotFoundError: shape: (1, 28)
┌──────────┬──────────┬─────┬─────┬─────┬─────┬─────┬───────────┬───────┐
│ estacion ┆ magnitud ┆ 1   ┆ 2   ┆ ... ┆ 23  ┆ 0   ┆ fecha     ┆ horas │
│ ---      ┆ ---      ┆ --- ┆ --- ┆     ┆ --- ┆ --- ┆ ---       ┆ ---   │
│ i64      ┆ i64      ┆ f64 ┆ f64 ┆     ┆ f64 ┆ f64 ┆ str       ┆ i64   │
╞══════════╪══════════╪═════╪═════╪═════╪═════╪═════╪═══════════╪═══════╡
│ 4        ┆ 1        ┆ 1.0 ┆ 1.0 ┆ ... ┆ 1.0 ┆ 1.0 ┆ 2022-12-1 ┆ 0     │
└──────────┴──────────┴─────┴─────┴─────┴─────┴─────┴───────────┴───────┘

> Error originated just after operation: '  DF ["estacion", "magnitud", "1", "2"]; PROJECT */28 COLUMNS; SELECTION: "None"
'
This operation could not be added to the plan.

In [50]:
df.head()

estacion,magnitud,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,0,fecha,horas
i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i64
4,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"""2022-12-1""",0
4,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,"""2022-12-2""",1
4,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,"""2022-12-3""",2
4,1,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"""2022-12-4""",3
4,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,"""2022-12-5""",4


In [52]:
df.with_column(f"{df.with_column('horas')[3]}")[3]

ColumnNotFoundError: shape: (1, 28)
┌──────────┬──────────┬─────┬─────┬─────┬─────┬─────┬───────────┬───────┐
│ estacion ┆ magnitud ┆ 1   ┆ 2   ┆ ... ┆ 23  ┆ 0   ┆ fecha     ┆ horas │
│ ---      ┆ ---      ┆ --- ┆ --- ┆     ┆ --- ┆ --- ┆ ---       ┆ ---   │
│ i64      ┆ i64      ┆ f64 ┆ f64 ┆     ┆ f64 ┆ f64 ┆ str       ┆ i64   │
╞══════════╪══════════╪═════╪═════╪═════╪═════╪═════╪═══════════╪═══════╡
│ 4        ┆ 1        ┆ 2.0 ┆ 2.0 ┆ ... ┆ 1.0 ┆ 1.0 ┆ 2022-12-4 ┆ 3     │
└──────────┴──────────┴─────┴─────┴─────┴─────┴─────┴───────────┴───────┘

> Error originated just after operation: '  DF ["estacion", "magnitud", "1", "2"]; PROJECT */28 COLUMNS; SELECTION: "None"
'
This operation could not be added to the plan.